In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model

In [2]:
data = pd.read_csv('Churn_Modelling.csv')

In [3]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)
with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo = pickle.load(file)
with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [5]:
data['Gender'] = label_encoder_gender.transform(data['Gender'])

In [6]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [25]:
geo_encoded = onehot_encoder_geo.transform(data[['Geography']]).toarray()
geo_encoded

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [24]:
print(type(data['Geography']), type(data[['Geography']]))

<class 'pandas.core.series.Series'> <class 'pandas.core.frame.DataFrame'>


In [26]:
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [27]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [29]:
data = pd.concat([data.drop(['Geography'],axis = 1), geo_encoded_df], axis = 1)

In [30]:
## Splitting of data
X = data.drop(['EstimatedSalary'],axis = 1)
Y = data['EstimatedSalary']

from sklearn.model_selection import train_test_split
X_train, x_test, Y_train, y_test = train_test_split(X,Y,test_size = 0.2, random_state=42)

In [31]:
X_train = scaler.transform(X_train)
x_test = scaler.transform(x_test)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- CustomerId
- Exited
- RowNumber
- Surname
Feature names seen at fit time, yet now missing:
- EstimatedSalary


In [33]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
x_test = scaler.transform(x_test)

ValueError: could not convert string to float: "P'an"

In [35]:
X_train = X_train.drop(['Surname'],axis=1)
x_test = x_test.drop(['Surname'],axis=1)

In [36]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
x_test = scaler.transform(x_test)

In [41]:
X_train.shape

(8000, 14)

## ANN Model

In [40]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense


In [46]:
model = Sequential([
    Dense(units=64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(units=32, activation='relu'),
    Dense(units=1)
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics = ['mse'])

d:\jai\Python-Workspace\Krish Naik Udemy\Machine-Learning\12-Deep Learning\Bank_Employee_Retention_project\venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [47]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
earlyStopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)  

In [48]:
import datetime
log_dir = "regression_logs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [49]:
training = model.fit(
    X_train,
    Y_train,
    epochs=1000,
    validation_data=(x_test, y_test),
    callbacks=[earlyStopping, tensorboard]
)

Epoch 1/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 13385832448.0000 - mse: 13385832448.0000 - val_loss: 13015788544.0000 - val_mse: 13015788544.0000
Epoch 2/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 13271640064.0000 - mse: 13271640064.0000 - val_loss: 12788966400.0000 - val_mse: 12788966400.0000
Epoch 3/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 12867194880.0000 - mse: 12867194880.0000 - val_loss: 12199370752.0000 - val_mse: 12199370752.0000
Epoch 4/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 12049998848.0000 - mse: 12049998848.0000 - val_loss: 11185015808.0000 - val_mse: 11185015808.0000
Epoch 5/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 10817320960.0000 - mse: 10817320960.0000 - val_loss: 9803419648.0000 - val_mse: 9803419648.0000
Epoch 6/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 9277762560.0000 - mse: 9277762560.0000 - val_loss: 8218474496.0000 - val_mse: 8218474496.0000
Epoch 7/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6

In [50]:
model.evaluate(x_test, y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3350646528.0000 - mse: 3350646528.0000


[3350646528.0, 3350646528.0]

In [51]:
%tensorboard --logdir=regression_logs/fit/20250815-13196

UsageError: Line magic function `%tensorboard` not found.
